In [1]:
import torch
from torchvision import transforms
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()

import cv2, os
import skimage.io as io
from PIL import Image

# ignoring warnings
import warnings

warnings.simplefilter("ignore")
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import torch.nn.functional as F

import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader


In [2]:
# !aws s3 sync shopee-data s3://mk-dl-data

In [3]:
path = 'shopee-data'
dataset = pd.read_csv(path + '/merged_with_data_tensors.csv')
dataset = dataset.drop(columns=['Unnamed: 0'])

In [4]:
from torch.utils.data import Dataset
'''
    It creates the pairs of images for inputs, same image label = 1, vice versa
'''
class ShopeeImageDataset(Dataset):
    def __init__(self, path, training = False):
        self.path = path
        self.dataset = pd.read_csv(path)
        self.dataset = self.dataset.drop(columns=['Unnamed: 0'])
        self.setSize = self.dataset.shape[0]
        self.training = training
        
    def __len__(self):
        return self.setSize
    def __getitem__(self, idx):
        image1 = torch.load(self.dataset['tensor_1'][idx])
        image2 = torch.load(self.dataset['tensor_2'][idx])
        label_value = self.dataset['label'][idx]
        label = torch.tensor(label_value).long()
        return image1, image2, label

In [5]:
class ShopeeImagePairDataset(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file: 'shopee-data/merged_with_data_tensors.csv'
            root_dir (string): Directory with all the images: 'shopee-data/train_images'
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.dataframe = pd.read_csv(csv_file)
        self.root_dir = root_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img1_path = os.path.join(self.root_dir, self.dataframe['image_1'][idx])
        img2_path = os.path.join(self.root_dir, self.dataframe['image_2'][idx])
        
        image1 = self.transform(img1_path)
        image2 = self.transform(img2_path)
        
        label = torch.tensor(self.dataframe['label'][idx])
        return image1, image2, label.int()
    
    def transform(self, image_path):
        image = io.imread(image_path)
        dim = (200, 200)
        resized = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
        tran = transforms.ToTensor()  # Convert the numpy array (C, H, W) Tensor format and /255 normalize to [0, 1.0]
        img_tensor = tran(resized) # (C,H,W), channel order (B, G, R)
        return img_tensor

In [6]:
device = 'cpu'#'cuda' if torch.cuda.is_available() else 'cpu'
image_path = path + '/merged_with_data_tensors.csv'
shopee_dataset = ShopeeImageDataset(image_path)
batch_size = 4
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(shopee_dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(shopee_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = torch.utils.data.DataLoader(shopee_dataset, batch_size=batch_size,
                                                sampler=valid_sampler)
print(len(train_loader))
print(len(validation_loader))

26
7


In [48]:

# training and validation loss were calculated after every epoch
def train(model, train_loader, val_loader, num_epochs):
    train_losses = []
    val_losses = []
    cur_step = 0
    for epoch in range(num_epochs):
        running_loss = 0.0
        model.train()
        print("Starting epoch " + str(epoch+1))
        for i, data in enumerate(train_loader,0):
            img0, img1, label = data
            img0, img1, label = img0.to(device), img1.to(device) , label.to(device)
            optimizer.zero_grad()
             # Forward
            target = model(img0,img1)  
            target = target.squeeze(1)
            loss = criterion(target, label.float())
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        val_running_loss = 0.0
        
        #check validation loss after every epoch
        with torch.no_grad():
            model.eval()
            for i, data in enumerate(val_loader,0):
                img1, img2, label = data
                img1 = img1.to(device)
                img2 = img2.to(device)
                label = label.to(device)       
                target = model(img1,img2)  
                target = target.squeeze(1)
                loss = criterion(target, label.float())
                val_running_loss += loss.item()
                ps = torch.exp(target)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                
        avg_val_loss = val_running_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
        print('Epoch [{}/{}],Train Loss: {:.4f}, Valid Loss: {:.8f}'
            .format(epoch+1, num_epochs, avg_train_loss, avg_val_loss))
    print("Finished Training")  
    return train_losses, val_losses 

In [36]:
class ShopeeImageNet(nn.Module):
    def __init__(self):
        super(ShopeeImageNet, self).__init__()
        
        # Conv2d(input_channels, output_channels, kernel_size)
        self.conv1 = nn.Conv2d(3, 64, 3) 
        self.conv2 = nn.Conv2d(64, 128, 3)  
        self.conv3 = nn.Conv2d(128, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        self.bn3 = nn.BatchNorm2d(128)
        self.bn4 = nn.BatchNorm2d(256)
        self.dropout1 = nn.Dropout(0.1)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 21 * 21, 200)
        self.fcOut = nn.Linear(200, 1)
        self.sigmoid = nn.Sigmoid()
    
    def convs(self, x): 
        #32, 3, 200, 200
#         print(1, x.shape)
        x = F.relu(self.bn1(self.conv1(x)))
        # 32, 64, 198, 198
#         print(2, x.shape)
        x = F.max_pool2d(x, (2,2))
#         print(3, x.shape)
        # 32, 64, 99, 99
        x = F.relu(self.bn2(self.conv2(x)))
#         print(4, x.shape)
        # 32, 128, 97, 97
        x = F.max_pool2d(x, (2,2))
#         print(5, x.shape)
        # 32, 128, 48, 48
        x = F.relu(self.bn3(self.conv3(x)))
#         print(6, x.shape)
        # 32, 128, 46, 46
        x = F.max_pool2d(x, (2,2))
#         print(7, x.shape)
        # 32, 128, 23, 23
        x = F.relu(self.bn4(self.conv4(x)))
#         print(8, x.shape)
        # 32, 256, 21, 21
        return x

    def forward(self, x1, x2):        
        x1 = self.convs(x1)
#         print('forward')
#         print(9, x1.shape)
        x1 = x1.view(-1, 256 * 21 * 21)
#         print(10, x1.shape)
        x1 = self.sigmoid(self.fc1(x1))
#         print(11, x1.shape)
        x2 = self.convs(x2)
#         print(12, x2.shape)
        x2 = x2.view(-1, 256 * 21 * 21)
#         print(13, x2.shape)
        x2 = self.sigmoid(self.fc1(x2))
#         print(14, x2.shape)
        x = torch.abs(x1 - x2)
        x = self.fcOut(x)
#         print(15, x.shape)
        return x

In [9]:
# class SiameseNetwork(nn.Module):
#     def __init__(self):
#         super(SiameseNetwork, self).__init__()
#         self.cnn1 = nn.Sequential(
#             nn.ReflectionPad2d(1),
#             nn.Conv2d(3, 4, kernel_size=3),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm2d(4),
            
#             nn.ReflectionPad2d(1),
#             nn.Conv2d(4, 8, kernel_size=3),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm2d(8),


#             nn.ReflectionPad2d(1),
#             nn.Conv2d(8, 8, kernel_size=3),
#             nn.ReLU(inplace=True),
#             nn.BatchNorm2d(8),

#         )

#         self.fc1 = nn.Sequential(
#             nn.Linear(8*100*100, 500),
#             nn.ReLU(inplace=True),

#             nn.Linear(500, 500),
#             nn.ReLU(inplace=True),

#             nn.Linear(500, 5))

#     def forward_once(self, x):
#         print(1, x.shape)
#         output = self.cnn1(x)
#         print(2, output.shape)
#         output = output.view(output.size()[0], -1)
#         print(3, output.shape)
#         output = self.fc1(output)
#         print(4, output.shape)
#         return output

#     def forward(self, input1, input2):
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)
#         return output1, output2

In [43]:
net = ShopeeImageNet().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()
train(net, train_loader, validation_loader, 15)

Starting epoch 1
Epoch [1/15],Train Loss: 0.6877, Valid Loss: 0.67333959
Starting epoch 2
Epoch [2/15],Train Loss: 0.5189, Valid Loss: 0.56731595
Starting epoch 3
Epoch [3/15],Train Loss: 0.4340, Valid Loss: 0.50313544
Starting epoch 4
Epoch [4/15],Train Loss: 0.3956, Valid Loss: 0.49347380
Starting epoch 5
Epoch [5/15],Train Loss: 0.3480, Valid Loss: 0.50378957
Starting epoch 6
Epoch [6/15],Train Loss: 0.3445, Valid Loss: 0.47681679
Starting epoch 7
Epoch [7/15],Train Loss: 0.3222, Valid Loss: 0.43339978
Starting epoch 8
Epoch [8/15],Train Loss: 0.2988, Valid Loss: 0.41472725
Starting epoch 9
Epoch [9/15],Train Loss: 0.2602, Valid Loss: 0.44721632
Starting epoch 10
Epoch [10/15],Train Loss: 0.2555, Valid Loss: 0.43940586
Starting epoch 11
Epoch [11/15],Train Loss: 0.2549, Valid Loss: 0.43719989
Starting epoch 12
Epoch [12/15],Train Loss: 0.2162, Valid Loss: 0.43533286
Starting epoch 13
Epoch [13/15],Train Loss: 0.2006, Valid Loss: 0.42871889
Starting epoch 14
Epoch [14/15],Train Loss:

([0.687726399073234,
  0.518912217937983,
  0.4339963117471108,
  0.39564528373571545,
  0.3480041812245662,
  0.3445183098889314,
  0.322186641395092,
  0.29884947205965334,
  0.2602346489349237,
  0.25550638368496525,
  0.2548892629834322,
  0.21621090918779373,
  0.2006189587454383,
  0.18425898966737664,
  0.17538555338978767],
 [0.6733395883015224,
  0.5673159531184605,
  0.5031354427337646,
  0.4934737980365753,
  0.5037895696503776,
  0.4768167861870357,
  0.433399783713477,
  0.4147272514445441,
  0.44721631918634686,
  0.43940586277416777,
  0.4371998906135559,
  0.4353328560079847,
  0.42871889046260286,
  0.3970305749348232,
  0.42330577969551086])

In [44]:
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9)
criterion = nn.BCEWithLogitsLoss()
train(net, train_loader, validation_loader, 15)

Starting epoch 1
Epoch [1/15],Train Loss: 0.1751, Valid Loss: 0.44071536
Starting epoch 2
Epoch [2/15],Train Loss: 0.1555, Valid Loss: 0.41064913
Starting epoch 3
Epoch [3/15],Train Loss: 0.1359, Valid Loss: 0.39049576
Starting epoch 4
Epoch [4/15],Train Loss: 0.1169, Valid Loss: 0.43083871
Starting epoch 5
Epoch [5/15],Train Loss: 0.0807, Valid Loss: 0.40189452
Starting epoch 6
Epoch [6/15],Train Loss: 0.0665, Valid Loss: 0.30421914
Starting epoch 7
Epoch [7/15],Train Loss: 0.0741, Valid Loss: 0.48520387
Starting epoch 8
Epoch [8/15],Train Loss: 0.0615, Valid Loss: 0.46693171
Starting epoch 9
Epoch [9/15],Train Loss: 0.0453, Valid Loss: 0.43519041
Starting epoch 10
Epoch [10/15],Train Loss: 0.0449, Valid Loss: 0.49608188
Starting epoch 11
Epoch [11/15],Train Loss: 0.0313, Valid Loss: 0.45900984
Starting epoch 12
Epoch [12/15],Train Loss: 0.0420, Valid Loss: 0.42668013
Starting epoch 13
Epoch [13/15],Train Loss: 0.0281, Valid Loss: 0.43372375
Starting epoch 14
Epoch [14/15],Train Loss:

([0.1751433229073882,
  0.15554157042732605,
  0.13594171016190487,
  0.11685600855316107,
  0.08072082773567392,
  0.06648098999777666,
  0.07409354725566047,
  0.061485446219404154,
  0.045321206199542545,
  0.044925578646003626,
  0.03127374560258781,
  0.04195188661105931,
  0.028055404859165158,
  0.038401840174069196,
  0.03652631083521275],
 [0.44071535978998455,
  0.41064912719385965,
  0.39049576435770306,
  0.43083871262414114,
  0.4018945183072771,
  0.3042191434651613,
  0.4852038707051958,
  0.4669317070926939,
  0.4351904051644461,
  0.49608188228947775,
  0.45900984161666464,
  0.4266801306179592,
  0.4337237541164671,
  0.4052855691739491,
  0.4159452276570456])

In [46]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
criterion = nn.BCEWithLogitsLoss()
train(net, train_loader, validation_loader, 15)

Starting epoch 1
Epoch [1/15],Train Loss: 0.6623, Valid Loss: 0.50611464
Starting epoch 2
Epoch [2/15],Train Loss: 0.6016, Valid Loss: 0.51797582
Starting epoch 3
Epoch [3/15],Train Loss: 0.6087, Valid Loss: 0.48111221
Starting epoch 4
Epoch [4/15],Train Loss: 0.6440, Valid Loss: 0.49286142
Starting epoch 5
Epoch [5/15],Train Loss: 0.6840, Valid Loss: 0.46513806
Starting epoch 6
Epoch [6/15],Train Loss: 0.6602, Valid Loss: 0.48601155
Starting epoch 7
Epoch [7/15],Train Loss: 0.6020, Valid Loss: 0.40842569
Starting epoch 8
Epoch [8/15],Train Loss: 0.6171, Valid Loss: 0.37595067
Starting epoch 9
Epoch [9/15],Train Loss: 0.6199, Valid Loss: 0.45864494
Starting epoch 10
Epoch [10/15],Train Loss: 0.6132, Valid Loss: 0.47319835
Starting epoch 11
Epoch [11/15],Train Loss: 0.5801, Valid Loss: 0.52701383
Starting epoch 12
Epoch [12/15],Train Loss: 0.6115, Valid Loss: 0.42323677
Starting epoch 13
Epoch [13/15],Train Loss: 0.5766, Valid Loss: 0.62392559
Starting epoch 14
Epoch [14/15],Train Loss:

([0.6622893740781225,
  0.6016094420964901,
  0.6087492681466616,
  0.644046235543031,
  0.6840481483019315,
  0.6601663203193591,
  0.6020230295566412,
  0.6170969909200301,
  0.6199173285410955,
  0.6131548308409177,
  0.5801070773830781,
  0.6114575094901599,
  0.576627946816958,
  0.6186155241269332,
  0.5615347142402942],
 [0.5061146361487252,
  0.5179758157048907,
  0.48111221407141,
  0.49286141778741566,
  0.4651380607060024,
  0.4860115477016994,
  0.4084256887435913,
  0.37595067279679434,
  0.45864494144916534,
  0.47319834785802023,
  0.5270138340336936,
  0.4232367681605475,
  0.6239255879606519,
  0.4964679905346462,
  0.41553084339414326])

In [47]:
optimizer = torch.optim.Adam(net.parameters(), lr=0.005)
criterion = nn.BCEWithLogitsLoss()
train(net, train_loader, validation_loader, 15)

Starting epoch 1
Epoch [1/15],Train Loss: 0.5687, Valid Loss: 0.50100323
Starting epoch 2
Epoch [2/15],Train Loss: 0.5553, Valid Loss: 0.56828134
Starting epoch 3
Epoch [3/15],Train Loss: 0.5140, Valid Loss: 0.47692915
Starting epoch 4
Epoch [4/15],Train Loss: 0.5679, Valid Loss: 0.42278424
Starting epoch 5
Epoch [5/15],Train Loss: 0.5538, Valid Loss: 0.54363891
Starting epoch 6
Epoch [6/15],Train Loss: 0.5400, Valid Loss: 0.43242516
Starting epoch 7
Epoch [7/15],Train Loss: 0.6109, Valid Loss: 0.56211643
Starting epoch 8
Epoch [8/15],Train Loss: 0.6179, Valid Loss: 0.66982604
Starting epoch 9
Epoch [9/15],Train Loss: 0.7160, Valid Loss: 0.53676099
Starting epoch 10
Epoch [10/15],Train Loss: 0.6780, Valid Loss: 0.60681472
Starting epoch 11
Epoch [11/15],Train Loss: 0.7104, Valid Loss: 0.44662353
Starting epoch 12
Epoch [12/15],Train Loss: 0.6693, Valid Loss: 0.47479344
Starting epoch 13
Epoch [13/15],Train Loss: 0.6031, Valid Loss: 0.47811382
Starting epoch 14
Epoch [14/15],Train Loss:

([0.5686799677518698,
  0.5552784066933852,
  0.5139550205606681,
  0.5678679530437176,
  0.5538326605008199,
  0.5400155438826635,
  0.6109110071108892,
  0.6179086886919462,
  0.7159804896666453,
  0.6780402832306348,
  0.7103562068480712,
  0.6693481734165778,
  0.6030816023166363,
  0.5504048415101491,
  0.451857885489097],
 [0.5010032270635877,
  0.5682813440050397,
  0.4769291451999119,
  0.42278424331120085,
  0.5436389063085828,
  0.43242516262190683,
  0.5621164270809719,
  0.6698260392461505,
  0.5367609901087624,
  0.6068147165434701,
  0.4466235339641571,
  0.47479343840054106,
  0.47811382157461985,
  0.39480319193431307,
  0.31454537117055487])